<a href="https://colab.research.google.com/github/m-mehabadi/grad-maker/blob/main/_notebooks/UniversalAttackGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Areas:
* Domain Generalization in Classification
* Domain Generalization in Mitosis Detection
* Universal Adversarial Example Generator
* An Aproach to Optimize Robustness and Performance Simultaneously
* Multi-Tasking using GradMaker
* Applications in Federated Learning


In [ ]:
!pip install tqdm

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

import gc
import torch.nn.functional as F
from torch.autograd import Variable

from tqdm.auto import tqdm

In [ ]:
# config
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Param
num_epochs = 10
num_classes = 10
batch_size = 200
learning_rate = 0.001

In [ ]:
# MNIST datasets
train_dataset = torchvision.datasets.MNIST(root='./', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./', train=False, transform=transforms.ToTensor(), download=True)

# MNIST dataloaders
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# CNN(two layer)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7 * 7 * 32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
def validate(model, dloader, pert=None, is_batched=True):
    model.eval()
    correct = 0
    total = 0
    pert_ = pert(model, dloader) if (not is_batched) and (pert is not None) else None
    for i, (images, labels) in enumerate(tqdm(dloader)):
        images = images.to(device)
        labels = labels.to(device)
        
        #
        if pert is not None:
            images = Variable(images.clone(), requires_grad=True)
            pert_ = pert(model, images, labels) if is_batched else pert_
            images = images.data.detach() + pert_.data.detach()
        #
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        #
        # print(f"Batch {i}/{len(dloader)}, processed.")

        gc_collect(pert_, images, labels, outputs, predicted, _)
        empty_cache()
    return (100 * correct / total)

In [ ]:
model = ConvNet(num_classes).to(device)

# Loss and optimize
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train
model.train()
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backprop and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print (f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_step}], Training Loss: {loss.item():.4f}')

# Test the model
print(f'Model Accuracy on the 10000 test images: {validate(model, test_loader):.4f} %')

Epoch [1/10], Step [100/300], Training Loss: 0.2070
Epoch [1/10], Step [200/300], Training Loss: 0.0764
Epoch [1/10], Step [300/300], Training Loss: 0.0746
Epoch [2/10], Step [100/300], Training Loss: 0.0139
Epoch [2/10], Step [200/300], Training Loss: 0.0774
Epoch [2/10], Step [300/300], Training Loss: 0.0449
Epoch [3/10], Step [100/300], Training Loss: 0.0166
Epoch [3/10], Step [200/300], Training Loss: 0.0215
Epoch [3/10], Step [300/300], Training Loss: 0.0162
Epoch [4/10], Step [100/300], Training Loss: 0.0095
Epoch [4/10], Step [200/300], Training Loss: 0.0157
Epoch [4/10], Step [300/300], Training Loss: 0.0432
Epoch [5/10], Step [100/300], Training Loss: 0.0168
Epoch [5/10], Step [200/300], Training Loss: 0.0098
Epoch [5/10], Step [300/300], Training Loss: 0.0612
Epoch [6/10], Step [100/300], Training Loss: 0.0268
Epoch [6/10], Step [200/300], Training Loss: 0.0106
Epoch [6/10], Step [300/300], Training Loss: 0.0028
Epoch [7/10], Step [100/300], Training Loss: 0.0088
Epoch [7/10]

  0%|          | 0/50 [00:00<?, ?it/s]

Model Accuracy on the 10000 test images: 99.0200 %


### Now let's generate the attack


In [ ]:
def gc_collect(*vars):
    for var in vars:
        del var
    gc.collect()

def empty_cache():
    torch.cuda.empty_cache()

In [ ]:
def adv_clip(adv, real, alpha=0):
    return torch.clamp(adv, min=torch.clamp(real-alpha, min=0.),
                       max=torch.clamp(real+alpha, max=1.))

In [ ]:
def accuracy(model, X, y):
    outputs = model(X)
    _, predicted = torch.max(outputs.data, 1)
    total = y.size(0)
    correct = (predicted == y).sum().item()
    return correct / total

In [ ]:
def gradient_maker(domain_grads, epsilon=0.5, alpha=0.05, alpha_schedule='exp'):
    iters = 10000
    dgr = domain_grads.view(domain_grads.shape[0], -1).to(device)
    number_of_domains, dim = dgr.shape

    #
    g = torch.randn(dim).to(device)
    u_ = torch.zeros(number_of_domains).to(device)
    
    iter = 0
    while (not torch.abs(torch.min(dgr@g)-epsilon)<=0.001) and (iter < iters):
        u_ = u_ + alpha*(epsilon - (dgr@g))
        g = (1./number_of_domains)*torch.sum(((1+(u_>=0)*u_).reshape(number_of_domains, 1))*dgr, axis=0)
        iter += 1
    return g.view(domain_grads.shape[1:])

In [ ]:
def dX(model, X, y):
    X = Variable(X.data, requires_grad=True)
    outputs = model(X)
    loss = F.cross_entropy(outputs, y)
    loss.backward()
    return X.grad.data.clone()

def fgsm(model, X, y, eps=0.1):
    advs = X + eps * torch.sign(dX(model, X, y))
    return adv_clip(advs, X, alpha=eps) - X

def universal_v1(model, X, y, alpha=0.05, steps=2, eps=0.15):
    X_first = X.clone()
    for step in range(steps):
        X_grad = dX(model, X, y)
        X_advs = adv_clip(X + alpha * torch.sign(gradient_maker(X_grad)), X, eps)
        X = Variable(X_advs.data, requires_grad=False)
        
        #
        gc_collect(X_advs, X_grad)
        empty_cache()
    return X - X_first

def universal_v2(model, dataloader, alpha=0.05, steps=2, eps=0.15):
    pert = None
    for step in range(steps):
        grads = []
        for i, (images, labels) in enumerate(tqdm(dataloader)):
            images, labels = images.to(device), labels.to(device)
            if pert is None:
                pert = torch.zeros_like(images)
            images_grads = dX(model, adv_clip(images + pert, images, eps), labels)
            grads.append(gradient_maker(images_grads))
        pert += alpha * torch.sign(gradient_maker(torch.stack((grads))))
    return pert

In [ ]:
validate(model, train_loader, fgsm, is_batched=True)

  0%|          | 0/300 [00:00<?, ?it/s]

51.69

In [ ]:
# let's do it manually to check if everything is alright

total = 0
correct = 0
for X, y in train_loader:
    X, y = X.to(device), y.to(device)
    X = Variable(X, requires_grad=True)
    X = X.data.detach() + fgsm(model, X, y).data.detach()
    outputs = model(X)
    _, predicted = torch.max(outputs.data, 1)
    total += y.size(0)
    correct += (predicted == y).sum().item()
print(correct/total)

0.5169


In [ ]:
validate(model, test_loader, fgsm, True)

  0%|          | 0/50 [00:00<?, ?it/s]

52.97

In [ ]:
validate(model, train_loader, universal_v2, False)

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

77.61666666666666